# Markov Chain Text Generation using NLP

This notebook is all about me trying to implement the fundamental mechnsm that is used in NLPs the probablity of conecpt of markov chain and building a simple Markov Chain model for text generation using n-grams and transition matrices.

## Overview
- **Markov Chain**: A stochastic model where the probability of each event depends only on the state attained in the previous event
- **N-grams**: Contiguous sequences of n items from a given sample of text
- **Transition Matrix**: A matrix that describes the transitions of a Markov chain

In [3]:
import re
from nltk import ngrams
import numpy as np

### Text Preprocessing

We start with a sample text and clean it by removing punctuation and converting to lowercase.

In [4]:
text = "I love cats. Cats are my favorite animal. I have two cats."
print(f"Original text: {text}")
text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
text = text.lower()
print(f"Cleaned text: {text}")

Original text: I love cats. Cats are my favorite animal. I have two cats.
Cleaned text: i love cats cats are my favorite animal i have two cats


### Generate N-grams

In [6]:
# Split text into individual words
words = text.split()
print(f"Words in text: {words}")
print(f"Number of words: {len(words)}")

# Generate n-grams (word pairs for n=2)
n = 2
n_grams = ngrams(words, n)
n_grams = list(n_grams)

print(f"\nGenerated bigrams (word pairs):")
print(n_grams)
print(f"Number of bigrams: {len(n_grams)}")

Words in text: ['i', 'love', 'cats', 'cats', 'are', 'my', 'favorite', 'animal', 'i', 'have', 'two', 'cats']
Number of words: 12

Generated bigrams (word pairs):
[('i', 'love'), ('love', 'cats'), ('cats', 'cats'), ('cats', 'are'), ('are', 'my'), ('my', 'favorite'), ('favorite', 'animal'), ('animal', 'i'), ('i', 'have'), ('have', 'two'), ('two', 'cats')]
Number of bigrams: 11


### Extract Unique Words

In [7]:
# Get unique words from our text (this will be our vocabulary)
unique_words = list(set(words))
print(f"Unique words in our vocabulary:")
print(unique_words)
print(f"Vocabulary size: {len(unique_words)} words")

# Show word to index mapping
print(f"\nWord to index mapping:")
for i, word in enumerate(unique_words):
    print(f"{word} -> {i}")

Unique words in our vocabulary:
['have', 'are', 'cats', 'two', 'i', 'my', 'animal', 'favorite', 'love']
Vocabulary size: 9 words

Word to index mapping:
have -> 0
are -> 1
cats -> 2
two -> 3
i -> 4
my -> 5
animal -> 6
favorite -> 7
love -> 8


### Build Transition Matrix

The transition matrix stores the count of how often each word follows another word.

In [8]:
# creating a zero matrix with dimensions (vocabulary_size x vocabulary_size)
transition_matrix = np.zeros((len(unique_words), len(unique_words)))
print(f"Transition matrix shape: {transition_matrix.shape}")

for i, word in enumerate(unique_words):
    for j, next_word in enumerate(unique_words):
        count = 0
        
        for n_gram in n_grams:
            if n_gram[0] == word and n_gram[1] == next_word:
                count += 1
        transition_matrix[i, j] = count

print(f"\nRaw transition matrix (counts):")
print(transition_matrix)


print(f"\nAnalyzing bigram transitions:")
for n_gram in n_grams:
    word1, word2 = n_gram
    i = unique_words.index(word1)
    j = unique_words.index(word2)
    print(f"{n_gram} -> count: {int(transition_matrix[i, j])}")

Transition matrix shape: (9, 9)

Raw transition matrix (counts):
[[0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]]

Analyzing bigram transitions:
('i', 'love') -> count: 1
('love', 'cats') -> count: 1
('cats', 'cats') -> count: 1
('cats', 'are') -> count: 1
('are', 'my') -> count: 1
('my', 'favorite') -> count: 1
('favorite', 'animal') -> count: 1
('animal', 'i') -> count: 1
('i', 'have') -> count: 1
('have', 'two') -> count: 1
('two', 'cats') -> count: 1


### Normalize Transition Matrix

Convert counts to probabilities by normalizing each row to sum to 1.

In [10]:
# Normalize the transition matrix to get probabilities
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

print(f"Normalized transition matrix (probabilities):")
print(transition_matrix)

print(f"\nTransition probabilities for each word:")
for i, word in enumerate(unique_words):
    print(f"Word '{word}' can be followed by:")
    for j, next_word in enumerate(unique_words):
        prob = transition_matrix[i, j]
        if prob > 0:
            print(f"  -> '{next_word}' with probability {prob:.2f}")
    print()

Normalized transition matrix (probabilities):
[[0.  0.  0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0. ]]

Transition probabilities for each word:
Word 'have' can be followed by:
  -> 'two' with probability 1.00

Word 'are' can be followed by:
  -> 'my' with probability 1.00

Word 'cats' can be followed by:
  -> 'are' with probability 0.50
  -> 'cats' with probability 0.50

Word 'two' can be followed by:
  -> 'cats' with probability 1.00

Word 'i' can be followed by:
  -> 'have' with probability 0.50
  -> 'love' with probability 0.50

Word 'my' can be followed by:
  -> 'favorite' with probability 1.00

Word 'animal' can be followed by:
  -> 'i' with probability 1.00

Word 'favorite' can be followed 

### Text Generation Function

In [11]:
def generate_text_markov(start_word, num_words=12, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    if start_word not in unique_words:
        print(f"Warning: '{start_word}' not in vocabulary. Available words: {unique_words}")
        return None
    
    current_word = start_word
    generated_text = current_word
    
    print(f"Starting with word: '{current_word}'")
    
    for step in range(num_words):
        
        current_word_index = unique_words.index(current_word)
        probabilities = transition_matrix[current_word_index]
        
        # Check if there are any possible next words
        if np.sum(probabilities) == 0:
            print(f"No possible next words for '{current_word}'. Stopping generation.")
            break
        
        # Select next word randomly based on probabilities
        next_word_index = np.random.choice(len(unique_words), p=probabilities)
        next_word = unique_words[next_word_index]
        
        print(f"Step {step + 1}: '{current_word}' -> '{next_word}' (prob: {probabilities[next_word_index]:.2f})")
        
        generated_text += " " + next_word
        
        # Set current word to next word for next iteration
        current_word = next_word
    
    return generated_text

### Generate Text Examples

In [12]:
# Example 1: Start with 'i'
print("example: Starting with 'i'")
result1 = generate_text_markov('i', num_words=12, seed=42)
print(f"\nGenerated text: {result1}")

example: Starting with 'i'
Starting with word: 'i'
Step 1: 'i' -> 'have' (prob: 0.50)
Step 2: 'have' -> 'two' (prob: 1.00)
Step 3: 'two' -> 'cats' (prob: 1.00)
Step 4: 'cats' -> 'cats' (prob: 0.50)
Step 5: 'cats' -> 'are' (prob: 0.50)
Step 6: 'are' -> 'my' (prob: 1.00)
Step 7: 'my' -> 'favorite' (prob: 1.00)
Step 8: 'favorite' -> 'animal' (prob: 1.00)
Step 9: 'animal' -> 'i' (prob: 1.00)
Step 10: 'i' -> 'love' (prob: 0.50)
Step 11: 'love' -> 'cats' (prob: 1.00)
Step 12: 'cats' -> 'cats' (prob: 0.50)

Generated text: i have two cats cats are my favorite animal i love cats cats


In [13]:
print(f"Available words in vocabulary: {unique_words}")
current_word = input("Enter your initial word to begin from: ")
if current_word in unique_words:
    result = generate_text_markov(current_word, num_words=12)
    print(f"\nFinal generated text: {result}")
else:
    print(f"Word '{current_word}' not found in vocabulary!")

Available words in vocabulary: ['have', 'are', 'cats', 'two', 'i', 'my', 'animal', 'favorite', 'love']
Starting with word: 'i'
Step 1: 'i' -> 'love' (prob: 0.50)
Step 2: 'love' -> 'cats' (prob: 1.00)
Step 3: 'cats' -> 'are' (prob: 0.50)
Step 4: 'are' -> 'my' (prob: 1.00)
Step 5: 'my' -> 'favorite' (prob: 1.00)
Step 6: 'favorite' -> 'animal' (prob: 1.00)
Step 7: 'animal' -> 'i' (prob: 1.00)
Step 8: 'i' -> 'have' (prob: 0.50)
Step 9: 'have' -> 'two' (prob: 1.00)
Step 10: 'two' -> 'cats' (prob: 1.00)
Step 11: 'cats' -> 'are' (prob: 0.50)
Step 12: 'are' -> 'my' (prob: 1.00)

Final generated text: i love cats are my favorite animal i have two cats are my


## Conclusion

well this learning notebook was to learn implementation of text generation using Markov chains:

### What I Observed:
- The model learns word transition patterns from the training text
- Words like 'cats' and 'i' have multiple possible next words (probabilistic)
- Most other words have deterministic transitions due to limited data
- Generated text follows the learned patterns but can create new combinations


- Limited training data leads to repetitive patterns